In [15]:
# Importing pandas library
import pandas as pd

# Importing datetime library
from datetime import datetime, timedelta, date

##### Reference

    Model:
    - https://towardsdatascience.com/predicting-next-purchase-day-15fae5548027

    Dataset:
    - https://www.kaggle.com/datasets/vijayuv/onlineretail

In [10]:
## Reading dataset
df_raw = pd.read_csv('OnlineRetail.csv', header=0, encoding='unicode_escape')  

In [12]:
# Convert "InvoiceDate" from string to datetime
df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

In [17]:
# Select only UK data
df_uk = df_raw.query("Country=='United Kingdom'").drop(columns=['Country'], axis=0).reset_index(drop=True)

In [23]:
# Selecting purchases before the cut-off date
df_uk_6m = df_uk[(df_uk['InvoiceDate'] <  datetime(2011,9,1)) & (df_uk['InvoiceDate'] >= datetime(2011, 3,1))].reset_index(drop=True)

# Selecting purchases after the cut-off date 
df_uk_nxt = df_uk[(df_uk['InvoiceDate'] >= datetime(2011,9,1)) & (df_uk['InvoiceDate']  < datetime(2011,12,1))].reset_index(drop=True)

In [21]:
# Selecting distinct users
df_uk_users = df_uk[['CustomerID']].drop_duplicates()

In [33]:
# Using the table df_uk_nx to calculate the number of days until the next purchase
from pyparsing import col


df_nxt_purch = df_uk_nxt[['CustomerID', 'InvoiceDate']]\
                .groupby(['CustomerID']).min()\
                .reset_index()\
                .rename(columns={'InvoiceDate':'NextPurchaseDate'})

# Using the table df_uk_6m to calculate the last purchase date
df_lst_purch = df_uk_6m[['CustomerID', 'InvoiceDate']]\
                .groupby(['CustomerID']).max()\
                .reset_index()\
                .rename(columns={'InvoiceDate':'LastPurchaseDate'})

# Merging purchase dates
df_purch_dt = df_uk_users\
                .merge(df_nxt_purch, how='left', on=['CustomerID'])\
                .merge(df_lst_purch, how='left', on=['CustomerID'])

# Calculate the time difference from the last purchase to the next purchase in days
df_purch_dt['DayUntilNextPurchase'] = (df_purch_dt['NextPurchaseDate'] - df_purch_dt['LastPurchaseDate']).dt.days

# Drop spare columns
df_purch_dt = df_purch_dt.drop(columns=['NextPurchaseDate', 'LastPurchaseDate'], axis=0)

# Fill missing values with 9999
df_purch_dt['DayUntilNextPurchase'] = df_purch_dt['DayUntilNextPurchase'].fillna(9999)